# Preprocessing
Created by Andy Smet, Kristof Egghe


In this file we load the data of the goodreads dataset. 
We also remove duplicates, remove users and items with a support less than user/itemMminsup and create consecutive IDs for the users that remain
Finally we will create a dataframe where we group the interactions on user_id and sort the interacted items on the date-time the user interacted with them.

Later we want to group the different volumes of one book series together, so we will make a csv of books_df with those volumes grouped for later use.

# Data Subset and Filepaths
In this file we load the data of the goodreads dataset. 
We also remove duplicates, remove users and items with a support less than user/itemMminsup and create consecutive IDs for the users and items that remain
Finally we will create a dataframe where we group the interactions on user_id and sort the interacted items on the date-time the user interacted with them.

In [1]:
# Use all data or a subset
allData=True
if allData:
  numberOfInteractions = -1
  userMinsup = 5
  itemMinsup = 35

else:
  numberOfInteractions = 1000000
  userMinsup = 20
  itemMinsup = 20


path="/home/andy/Desktop/AI_Project/"
# path='./drive/MyDrive/AI Project/'
# goodreads_path = './drive/MyDrive/AI Project/datasets/Goodreads/'
goodreads_path = path+"datasets/Goodreads/"
books = 'goodreads_books_comics_graphic.json.gz'
interactions = 'goodreads_interactions_comics_graphic.json.gz'
reviews = 'goodreads_reviews_comics_graphic.json.gz'

# Install Packages

In [2]:
# Install your required packages here
!pip install --user pandas numpy matplotlib sklearn 
import os
import time
import numpy as np
import pandas as pd
import sklearn
import gzip
import json
from tqdm import tqdm
import os
from collections import Counter
from datetime import datetime
import math
tqdm.pandas() #for progres_apply etc.

start = time.time()

You should consider upgrading via the '/home/andy/anaconda3/bin/python -m pip install --upgrade pip' command.


In [3]:
# #Mount google drive
# from google.colab import drive
# drive.mount('/content/drive')



# 1. Load Goodreads Data

In [4]:
#read file line-by-line and parse json, returns dataframe
# read_max = -1 to read full dataset
def parse_json(filename_gzipped_python_json, read_max=-1):
  #read gzipped content
  f=gzip.open(filename_gzipped_python_json,'r')
  
  #parse json
  parse_data = []
  for line in tqdm(f): #tqdm is for showing progress bar, always good when processing large amounts of data
    line = line.decode('utf-8')
    line = line.replace('true','True') #difference json/python
    line = line.replace('false','False')
    parsed_result = eval(line) #load python nested datastructure
    parse_data.append(parsed_result)
    if read_max !=-1 and len(parse_data) > read_max:
      print(f'Break reading after {read_max} records')
      break
  print(f"Reading {len(parse_data)} rows.")

  #create dataframe
  df= pd.DataFrame.from_dict(parse_data)
  return df


In [5]:
#books
books_df = pd.read_json(goodreads_path + books, lines=True)
books_df = books_df[['book_id',	'title','authors',	'publisher',	'num_pages',	'publication_year']]
display(books_df.head(5))

,book_id,title,authors,publisher,num_pages,publication_year
0,25742454,The Switchblade Mamma,"[{'author_id': '8551671', 'role': ''}]",,,
1,30128855,Cruelle,"[{'author_id': '3274315', 'role': ''}]",Dargaud,,2016
2,13571772,Captain America: Winter Soldier (The Ultimate ...,"[{'author_id': '37450', 'role': ''}]",Hachette Partworks Ltd.,146,2012
3,35452242,Bounty Hunter 4/3: My Life in Combat from Mari...,"[{'author_id': '16209952', 'role': ''}, {'auth...",,,
4,707611,"Superman Archives, Vol. 2","[{'author_id': '81563', 'role': ''}, {'author_...",DC Comics,272,1997


# Group volumes

In [6]:
import re


books_df_g = books_df.copy()
books_df_g["title"] = books_df_g["title"].str.replace(r'(\d)+ ', '')
books_df_g["title"] = books_df_g["title"].str.replace(r' (\d)+', '')
books_df_g["title"] = books_df_g["title"].str.replace(',', '')
display(books_df_g.title)
print("oki{}".format(len(books_df_g[books_df_g.title.str.split(pat = ' Volume| Vol\.| vol\.| #').str[0]==""])))
books_df_g = books_df_g.groupby(books_df_g.title.str.split(pat = ' Volume| Vol\.| vol\.| #').str[0])
print(books_df['book_id'].nunique())
print(books_df_g.ngroups)

books_df_g = books_df_g.agg({'book_id':list, 'title':lambda x: x.iloc[0],'authors':lambda x: x.iloc[0],	'publisher': list,	'num_pages':sum,	'publication_year':lambda x: x.iloc[0]})
books_df_g["publisher"] = books_df_g["publisher"].apply(set)

# print(len(books_df_g))

print(len(books_df_g))

<ipython-input-6-a70008762a31>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  books_df_g["title"] = books_df_g["title"].str.replace(r'(\d)+ ', '')
<ipython-input-6-a70008762a31>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  books_df_g["title"] = books_df_g["title"].str.replace(r' (\d)+', '')


0                                    The Switchblade Mamma
1                                                  Cruelle
2        Captain America: Winter Soldier (The Ultimate ...
3        Bounty Hunter/3: My Life in Combat from Marine...
4                                   Superman Archives Vol.
                               ...                        
89406                      Doctor Who: Free Comic Book Day
89407                                      Love: The Tiger
89408                    La búsqueda del pájaro del tiempo
89409                                      Dejah Thoris #1
89410                                         Dead Endings
Name: title, Length: 89411, dtype: object

oki0
89411
50365
50365


In [7]:
#get author names (authors metadata is an additional dowload from goodreads)
authors = '/metadata/' + 'goodreads_book_authors.json.gz'
authors_df =  pd.read_json(goodreads_path + authors, lines=True) #829.529 authors (also non-graphic and comics)
display(authors_df.head(5))

,average_rating,author_id,text_reviews_count,name,ratings_count
0,3.98,604031,7,Ronald J. Fields,49
1,4.08,626222,28716,Anita Diamant,546796
2,3.92,10333,5075,Barbara Hambly,122118
3,3.68,9212,36262,Jennifer Weiner,888522
4,3.82,149918,96,Nigel Pennick,1740


# 2. Clean Data

In [8]:
# add author name to books_df and safe the dataframes

author_id_to_name = {}
for idx, row in tqdm(authors_df.iterrows(), total=authors_df.shape[0]):
  author_id_to_name[row['author_id']] = row['name']
display(books_df.head(5))
#important: type of author is np.int64
books_df_g['author_name'] = books_df_g['authors'].apply(lambda authors_dct_lst: author_id_to_name.get(np.int64(authors_dct_lst[0]['author_id'])))
books_df_g["group_id"]=range(0,len(books_df_g))
books_df['author_name'] = books_df['authors'].apply(lambda authors_dct_lst: author_id_to_name.get(np.int64(authors_dct_lst[0]['author_id'])))

books_df_g.to_csv(path+"datasets/books_df_grouped.csv", index=False)
books_df.to_csv(path+"datasets/books_df_singular.csv", index=False)

display(books_df.head(5))

100%|██████████| 829529/829529 [01:43<00:00, 7990.31it/s]


,book_id,title,authors,publisher,num_pages,publication_year
0,25742454,The Switchblade Mamma,"[{'author_id': '8551671', 'role': ''}]",,,
1,30128855,Cruelle,"[{'author_id': '3274315', 'role': ''}]",Dargaud,,2016
2,13571772,Captain America: Winter Soldier (The Ultimate ...,"[{'author_id': '37450', 'role': ''}]",Hachette Partworks Ltd.,146,2012
3,35452242,Bounty Hunter 4/3: My Life in Combat from Mari...,"[{'author_id': '16209952', 'role': ''}, {'auth...",,,
4,707611,"Superman Archives, Vol. 2","[{'author_id': '81563', 'role': ''}, {'author_...",DC Comics,272,1997


,book_id,title,authors,publisher,num_pages,publication_year,author_name
0,25742454,The Switchblade Mamma,"[{'author_id': '8551671', 'role': ''}]",,,,Lindsey Schussman
1,30128855,Cruelle,"[{'author_id': '3274315', 'role': ''}]",Dargaud,,2016,Florence Dupre la Tour
2,13571772,Captain America: Winter Soldier (The Ultimate ...,"[{'author_id': '37450', 'role': ''}]",Hachette Partworks Ltd.,146,2012,Ed Brubaker
3,35452242,Bounty Hunter 4/3: My Life in Combat from Mari...,"[{'author_id': '16209952', 'role': ''}, {'auth...",,,,Jason Delgado
4,707611,"Superman Archives, Vol. 2","[{'author_id': '81563', 'role': ''}, {'author_...",DC Comics,272,1997,Jerry Siegel


In [9]:
interactions_df = parse_json(goodreads_path + interactions, read_max=numberOfInteractions) #Note: RAM issue if loading with pd.read_json, no issue with parse_json 

#1) parse date 
interactions_df_new = interactions_df[['user_id', 'book_id', 'rating', 'date_updated']]
format_str = '%a %b %d %H:%M:%S %z %Y' #see https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior
#test: datetime_object = datetime.strptime('Fri Jun 21 10:25:05 -0700 2013', format_str) 
interactions_df_new['date_updated'] = interactions_df_new['date_updated'].progress_apply(lambda s: np.datetime64(datetime.strptime(s,format_str)))

#2) sort on user_id, then date
interactions_df_new = interactions_df_new.sort_values(by=['user_id', 'date_updated'], ascending=[True,True])
display(interactions_df_new.head(20))


7347630it [06:49, 17963.94it/s]


Reading 7347630 rows.


  0%|          | 1/7347630 [00:00<391:47:40,  5.21it/s]<ipython-input-9-97d4600a7fa1>:7: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  interactions_df_new['date_updated'] = interactions_df_new['date_updated'].progress_apply(lambda s: np.datetime64(datetime.strptime(s,format_str)))
100%|██████████| 7347630/7347630 [04:36<00:00, 26586.61it/s]
<ipython-input-9-97d4600a7fa1>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interactions_df_new['date_updated'] = interactions_df_new['date_updated'].progress_apply(lambda s: np.datetime64(datetime.strptime(s,format_str)))


,user_id,book_id,rating,date_updated
1651325,00004584d524ec468619e81b176cc991,271199,4,2013-06-21 17:23:44
1651324,00004584d524ec468619e81b176cc991,287380,4,2013-06-21 17:24:05
1651322,00004584d524ec468619e81b176cc991,287381,4,2013-06-21 17:24:31
1651316,00004584d524ec468619e81b176cc991,287382,4,2013-06-21 17:25:05
1651314,00004584d524ec468619e81b176cc991,287388,3,2013-06-21 17:25:13
1651312,00004584d524ec468619e81b176cc991,287385,4,2013-06-21 17:25:29
1651311,00004584d524ec468619e81b176cc991,287383,4,2013-06-21 17:25:37
1651308,00004584d524ec468619e81b176cc991,287364,5,2013-06-21 17:26:36
1651307,00004584d524ec468619e81b176cc991,287368,4,2013-06-21 17:26:43
1651306,00004584d524ec468619e81b176cc991,287371,4,2013-06-21 17:26:52


# 3. Create consecutive ID's

In [10]:
#1) convert user uuid to consecutive integer ID's 
dct = {}
def map_to_consecutive_id(uuid):
  if uuid in dct:
    return dct[uuid]
  else:
    id = len(dct)
    dct[uuid] = id
    return id
interactions_df_new['user_id_int'] = interactions_df_new['user_id'].progress_apply(map_to_consecutive_id)


100%|██████████| 7347630/7347630 [00:10<00:00, 711635.06it/s]


In [11]:
import pickle
def preprocess_classic(df):
    """
    Goal: - Remove reconsumption items
          - Remove items that have less than minsup interactions 
          - Remove users that have less than minsup interactions 
               
    :input df: Dataframe containing user_id, item_id and time
    """
    before = df.shape[0]
    #drop reconsumption items
    df = df.drop_duplicates(subset=["user_id","item_id"])
    print("After drop_duplicates (reconsumption items): {} -> {}".format(before,df.shape[0]))
    #compute item/user counts
    g1 = df.groupby('item_id', as_index=False)['user_id'].size()
    g1 = g1.rename({'size': 'count_item'}, axis='columns')

    df = pd.merge(df, g1, how='left', on=['item_id'])
    display(df.head(5))
    #drop items occurring less than minsup times
    before = df.shape[0]
    df = df[df['count_item'] >= itemMinsup]
    print("After dropping items with less than {} interactions: {} -> {}".format(itemMinsup, before,df.shape[0]))
    before = df.shape[0]
    #drop users with less then minsup items in history
    #changed minsup to minsup/2 so more users remain which is positive for the algorithm
    g2 = df.groupby('user_id', as_index=False)['item_id'].size()
    g2 = g2.rename({'size': 'count_user'}, axis='columns')
    df = pd.merge(df, g2, how='left', on=['user_id'])
    df = df[df['count_user'] >= userMinsup]
    df = df[['user_id','item_id','datetime']]
    print("After dropping users with less than {} interactions: {} -> {}".format(userMinsup, before,df.shape[0]))
    return df

#Remark: ignoring rating, considering all reviews as implicit possitive feedback
#print number of users and items
interactions_df_processed = interactions_df_new[['user_id_int','book_id','date_updated']]
interactions_df_processed = interactions_df_processed.rename(columns={"user_id_int": "user_id","book_id": "item_id", "date_updated": "datetime"})
print(f"number of unique users: {interactions_df_processed['user_id'].nunique()}")
print(f"number of unique items: {interactions_df_processed['item_id'].nunique()}")
interactions_df_processed = preprocess_classic(interactions_df_processed)
interactions_df_processed['item_id'] = interactions_df_processed['item_id'].astype('int32')
interactions_df_processed["item_id"]=interactions_df_processed["item_id"]
display(interactions_df_processed.head(5))
print(f"number of unique users: {interactions_df_processed['user_id'].nunique()}")
print(f"number of unique items: {interactions_df_processed['item_id'].nunique()}")
user_dic={}
counter=0
for i in interactions_df_processed['user_id'].unique():
  user_dic[i]=counter
  counter+=1

item_dic={}
counter=0
for i in interactions_df_processed['item_id'].unique():
  item_dic[i]=counter
  counter+=1

if allData:
  with open(path+"datasets/AllDataUser_dic.pickle", 'wb') as f:
      pickle.dump(user_dic, f, pickle.HIGHEST_PROTOCOL)
  with open(path+"datasets/AllDataItem_dic.pickle", 'wb') as f:
      pickle.dump(item_dic, f, pickle.HIGHEST_PROTOCOL)
  interactions_df_processed.to_csv(path+"datasets/AllDataPreprocessed_interactions.csv", index=False)
else:
  with open(path+"datasets/TestDataUser_dic.pickle", 'wb') as f:
      pickle.dump(user_dic, f, pickle.HIGHEST_PROTOCOL)
  with open(path+"datasets/TestDataItem_dic.pickle", 'wb') as f:
      pickle.dump(item_dic, f, pickle.HIGHEST_PROTOCOL)
  interactions_df_processed.to_csv(path+"datasets/TestDataPreprocessed_interactions.csv", index=False)


number of unique users: 342415
number of unique items: 89411
After drop_duplicates (reconsumption items): 7347630 -> 7347630


,user_id,item_id,datetime,count_item
0,0,271199,2013-06-21 17:23:44,10102
1,0,287380,2013-06-21 17:24:05,1628
2,0,287381,2013-06-21 17:24:31,198
3,0,287382,2013-06-21 17:25:05,247
4,0,287388,2013-06-21 17:25:13,249


After dropping items with less than 35 interactions: 7347630 -> 6686728
After dropping users with less than 5 interactions: 6686728 -> 6339989


,user_id,item_id,datetime
0,0,271199,2013-06-21 17:23:44
1,0,287380,2013-06-21 17:24:05
2,0,287381,2013-06-21 17:24:31
3,0,287382,2013-06-21 17:25:05
4,0,287388,2013-06-21 17:25:13


number of unique users: 143570
number of unique items: 26431


In [12]:
import random
sessions_df = interactions_df_processed.groupby(by='user_id', as_index=False)[['item_id','datetime']].agg(list)
display(sessions_df.head(10))

def split(items, percentage_train, seed):

  shuffledItems = items
  random.seed(seed)
  random.shuffle(shuffledItems)
  no_train_items = math.floor(len(shuffledItems) * percentage_train)
  return shuffledItems[0:no_train_items], shuffledItems[no_train_items:]

def trainTestSplit(df, percentage_train=0.8, seed=1):
  
  df['history'] = df['item_id'].apply(lambda items: split(items, percentage_train, seed)[0])
  df['future'] = df['item_id'].apply(lambda items: split(items, percentage_train, seed)[1])
  return df
for s in range(1,6):
  randomSplit_df = trainTestSplit(sessions_df, percentage_train=0.8, seed=s)
  if allData:
    randomSplit_df.to_csv(path+"datasets/AllDatafold"+str(s)+".csv",index=False)
  else:
    randomSplit_df.to_csv(path+"datasets/TestDatafold"+str(s)+".csv",index=False)

,user_id,item_id,datetime
0,0,"[271199, 287380, 287381, 287382, 287388, 28738...","[2013-06-21 17:23:44, 2013-06-21 17:24:05, 201..."
1,2,"[472331, 25103, 21330, 5805, 25100, 15719, 127...","[2014-06-30 02:30:37, 2014-06-30 02:30:44, 201..."
2,6,"[43070, 77727, 13324301, 15799936, 13623923, 6...","[2012-10-21 16:23:19, 2012-10-21 16:27:04, 201..."
3,11,"[9516, 9517, 9526, 9525, 103667, 1003419, 9214...","[2008-02-22 22:33:45, 2008-02-23 03:14:49, 200..."
4,14,"[707667, 31979, 22351, 653668, 176533, 503843,...","[2008-05-20 17:34:39, 2008-05-20 17:35:21, 200..."
5,18,"[10138607, 15197, 3052106, 18213224, 472331, 5...","[2013-11-10 22:27:53, 2013-11-10 22:30:35, 201..."
6,22,"[15789310, 17724775, 96358, 17899553, 3368983,...","[2016-02-11 22:25:57, 2016-02-11 22:43:58, 201..."
7,25,"[23197322, 23904736, 22553544, 11154863, 10689...","[2015-03-30 16:07:12, 2015-03-30 16:07:15, 201..."
8,26,"[9260507, 6593831, 9554949, 16108599, 12732062...","[2015-03-09 12:25:41, 2015-10-25 14:35:03, 201..."
9,29,"[13615, 13619, 13618, 13616, 13617, 13620, 136...","[2017-06-20 01:20:10, 2017-06-20 01:20:37, 201..."
